In [0]:
%tensorflow_version 2.x 

In [0]:
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.resnet50 import decode_predictions, preprocess_input

In [0]:
resnet_50 = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                       include_top=False,
                                       weights='imagenet')

In [0]:
!wget https://i.ytimg.com/vi/plFCDMQLGpw/hqdefault.jpg -O img.jpg





In [0]:
img = cv2.imread("img.jpg")[:,:,::-1] # strange syntax necessary to preserve color info
print(img.shape)

In [0]:
img = cv2.resize(img, (224, 224))
ax = plt.imshow(img)

In [0]:
def preprocess(img):
  # use the pre processing function of ResNet50 
  img = preprocess_input(img)
  
  # create a batch
  return tf.expand_dims(img, 0)

def preprocess2(img):
  img = img / 127.5
  img -= 1
  return tf.expand_dims(img, 0)

In [0]:
input_img = preprocess(img)
print(input_img.shape)

resnet_50_top = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                       include_top=True,
                                       weights='imagenet')

preds = resnet_50_top.predict(input_img)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [0]:
def get_activations(img, i):
  print(resnet_50)
  layer = resnet_50.layers[i]
  model = tf.keras.models.Model(inputs=resnet_50.inputs, outputs=layer.output)
  return model.predict(img)

In [0]:
acts = get_activations(input_img, 2)

In [0]:
def postprocess_activations(acts):
  #using the approach in https://arxiv.org/abs/1612.03928
  output = tf.abs(acts)
  output = tf.reduce_sum(output, axis=-1)
  output = tf.squeeze(output)

  #resize and convert to image 
  output = cv2.resize(output.numpy(), (224, 224))
  output /= tf.reduce_max(output)
  output *= 255
  return 255 - tf.cast(output, tf.uint8)

In [0]:
pa = postprocess_activations(acts)

In [0]:
def apply_heatmap(weights, img):
  heatmap = cv2.applyColorMap(weights, cv2.COLORMAP_JET)
  heatmap = cv2.addWeighted(heatmap, 0.7, img, 0.3, 0)
  return heatmap

In [0]:
 heatmap = apply_heatmap(pa.numpy(), img)

In [0]:
plt.axis('off')
ax = plt.imshow(heatmap)